# AI Model: Gemma by GOOGLE
We are going to use gemma as our pretrained, open source LLM. This model will be loaded in using 

In [1]:
pip install -q --upgrade keras-nlp

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q --upgrade keras  # Upgrade to Keras 3.

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


# Math Director Prompt:

You are the director of a mathematics solutions company;
you will take a given math problem, 
and break the task down into a list of individual steps that must be completed to solve the task;

for each math problem; you will delegate to a math solution assistant who will complete the task;

ONLY say "TERMINATE" after you have returned a solution for the problem.

In [15]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp
import kagglehub
import kaggle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown


class CFG:
    seed = 42
    dataset_path = "/AI Math Olympiad Prize"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train

keras.utils.set_random_seed(CFG.seed)

df1 = pd.read_csv("traincompiled.csv")
df2 = pd.read_csv("testcompiled.csv")
df = pd.concat([df1, df2], axis=0)
#df = df[:1000] # take first 1000 samples
df = df[:50]
df.head(10)


,Question,Level,Type,Solution,Answer
0,My school's math club has 6 boys and 8 girls. ...,Level 2,counting_and_probability,"With no restrictions, we are merely picking 6 ...",3003
1,How many 4-letter words with at least one cons...,Level 3,counting_and_probability,First we count the number of all 4-letter word...,609
2,"Let $p$ be the probability that, in the proces...",Level 5,counting_and_probability,Think of the problem as a sequence of H's and ...,37
3,How many four-digit numbers greater than 2999 ...,Level 4,counting_and_probability,"For the first digit, there are seven choices (...",4970
4,I have 5 marbles numbered 1 through 5 in a bag...,Level 3,counting_and_probability,There are $\binom{5}{2} = 10$ different pairs ...,6
5,The number of increasing sequences of positive...,Level 5,counting_and_probability,The numbers $a_i - i$ are ten not-necessarily ...,8
6,Find the sum of all integers $k$ such that $\b...,Level 4,counting_and_probability,"By Pascal's Identity, we have $\binom{23}{4} +...",24
7,Find the number of ordered pairs of positive i...,Level 5,counting_and_probability,There are $\left\lfloor\frac{999}{10}\right\rf...,738
8,Determine the number of ways to arrange the le...,Level 3,counting_and_probability,First we count the arrangements if all the let...,90
9,A mathematical organization is producing a set...,Level 5,counting_and_probability,"If a sequence contains no more than one 0, the...",372


In [16]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.Question,
                                                             solution=f"{row.Solution}\n\nAnswer:\n{row.Answer}"),
                                                             axis=1)
data = df.prompt.tolist()




ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html